In [2]:
from torchvision import transforms
from PIL import Image
from model import AggMInterface
from data import DInterface
import yaml
import numpy as np


# 指定checkpoint和模型超参数
config_path = "/media/cartolab3/DataDisk/wuqilong_file/Projects/RerenkVPR/logs/dinov2_backbone_dinov2_large/lightning_logs/version_0/hparams.yaml"
checkpoint_path = "/media/cartolab3/DataDisk/wuqilong_file/Projects/RerenkVPR/logs/dinov2_backbone_dinov2_large/lightning_logs/version_0/checkpoints/dinov2_backbone_epoch(14)_step(14655)_R1[0.9149]_R5[0.9622]_R10[0.9703].ckpt"

# 加载yaml文件
with open(config_path) as f:
    config = yaml.safe_load(f)

# 初始化data moudle
data_module = DInterface(**config)  # 数据相关配置
transform = data_module.valid_transform  # val transform

# 初始化model
model = AggMInterface.load_from_checkpoint(checkpoint_path)
model.eval()

正在尝试从以下路径导入模块: .dinov2_backbone
正在查找类: Dinov2Backbone


/home/cartolab3/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/cartolab3/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/home/cartolab3/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


AggMInterface(
  (model): Dinov2Backbone(
    (reduce_linear): Linear(in_features=1024, out_features=1024, bias=False)
    (model): DinoVisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
        (norm): Identity()
      )
      (blocks): ModuleList(
        (0-23): 24 x NestedTensorBlock(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): MemEffAttention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): LayerScale()
          (drop_path1): Identity()
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (act): GELU

In [17]:
from torchvision import transforms
import cv2
from einops import rearrange
from visualization.visualization_tools import get_pca_map
import numpy as np
import os
import matplotlib.pyplot as plt

# input_image_path
input_image_path = "/media/cartolab3/DataDisk/wuqilong_file/Projects/RerenkVPR/imgs/output_imgs/nordland/0000003.jpg"
image_size = (560,560)

def visualize_dinov2(input_image_path, image_size=(322,322)):
    # 定义加载图像并resize，输出tensor的transform
    ori_image = Image.open(input_image_path)
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor()
    ])

    image=transform(ori_image).unsqueeze(0).cuda()
    print(image.shape)

    # 输入模型
    cls_token, patch_tokens, (q, k, v) = model.forward(image,is_training=False,block_idx=-1)
    print(patch_tokens.shape)

    # 获取特征图
    features = patch_tokens.detach().cpu()
    features = rearrange(features, 'b (h w) c -> b h w c', h= int(image_size[0]/14), w=int(image_size[1]/14))
    features = features[0].float()

    # 设置重采样图大小
    color = get_pca_map(features, image_size, interpolation='bilinear')
    color=(color*255).astype(np.uint8)
    print(color.shape)

    # 将color代表的图像和原始图像拼接，形成一行两列的图像
    image = np.hstack((ori_image.resize(image_size), color))
    print(image.shape)

    # 定义输出文件名称
    output_file = f'{os.path.dirname(input_image_path)}/{os.path.basename(input_image_path).split(".")[0]}_dinov2_finetune.jpg'

    # cv2.imwrite(output_file, image)
    rgb_image = Image.fromarray(image)
    rgb_image.save(output_file)

In [19]:
import glob

# 可视化多张图像
img_path="./imgs/output_imgs/tokyo"
image_list = glob.glob(os.path.join(img_path, "*.jpg"))
print(len(image_list))
for image_path in image_list:
    if 'dinov2' not in image_path:
        visualize_dinov2(image_path,image_size=image_size)

6
torch.Size([1, 3, 560, 560])
torch.Size([1, 1600, 1024])
(560, 560, 3)
(560, 1120, 3)
torch.Size([1, 3, 560, 560])
torch.Size([1, 1600, 1024])
(560, 560, 3)
(560, 1120, 3)
